In [11]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import *
from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
from torchmetrics.classification import MulticlassF1Score
from torchinfo import summary

In [12]:
###  데이터 로딩
TEST_FILE='../data/mnist_test.csv'
TRAIN_FILE='../data/mnist_test.csv'


### CSV >>> DataFrame
TEST_DF = pd.read_csv(TEST_FILE)
TRAIN_DF = pd.read_csv(TRAIN_FILE)

In [13]:
TEST_DF

,0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,...,0.659,0.660,0.661,0.662,0.663,0.664,0.665,0.666,0.667,7
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,4
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9994,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
9995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3
9996,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,4
9997,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,5


In [14]:
TRAIN_DF.head()

,0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,...,0.659,0.660,0.661,0.662,0.663,0.664,0.665,0.666,0.667,7
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,4
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [15]:
# 학습용, 검증용, 테스트용 데이터 분리
X_train = TRAIN_DF[TRAIN_DF.columns[:-1]]
y_train = TRAIN_DF[TRAIN_DF.columns[-1:]]

X_test = TEST_DF[TEST_DF.columns[:-1]]
y_test = TEST_DF[TEST_DF.columns[-1:]]


# print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)/
X_train,X_val,y_train,y_val=train_test_split(X_train,y_train,random_state=0)

In [16]:
class MnistMCFModel(nn.Module):

    # 모델 구조 구성 및 인스턴스 생성 메서드
    def __init__(self):
        super().__init__()
        self.in_layer=nn.Linear(784, 64)
        self.hd_layer1=nn.Linear(64, 128)
        self.hd_layer2=nn.Linear(128, 256)
        self.hd_layer3=nn.Linear(256,512)
        self.hd_layer4=nn.Linear(512,128)
        self.hd_layer5=nn.Linear(128,64)
        self.out_layer=nn.Linear(64, 10)   # 다중분류 'Setosa', 'Versicolor', 'Virginica'

    # 순방향 학습 진행 메서드
    def forward(self, input_data):
        #- 입력층
        y=self.in_layer(input_data)
        y=F.relu(y)

        # - 은닉층 : 10개의 숫자 값(>=0)
        y=self.hd_layer1(y)
        y=F.relu(y)

        y=self.hd_layer2(y)
        y=F.relu(y)

        y=self.hd_layer3(y)
        y=F.relu(y)

        y=self.hd_layer4(y)
        y=F.relu(y)

        y=self.hd_layer5(y)
        y=F.relu(y)



        #- 출력층 : 5개의 숫자 값 => 다중 분류
        #- 손실함수 CrossEntropyLoss가 내부에서 softmax 진행
        return self.out_layer(y)

In [17]:
### [테스트] 모델 인스턴스 생성
model = MnistMCFModel()
print(model)

MnistMCFModel(
  (in_layer): Linear(in_features=784, out_features=64, bias=True)
  (hd_layer1): Linear(in_features=64, out_features=128, bias=True)
  (hd_layer2): Linear(in_features=128, out_features=256, bias=True)
  (hd_layer3): Linear(in_features=256, out_features=512, bias=True)
  (hd_layer4): Linear(in_features=512, out_features=128, bias=True)
  (hd_layer5): Linear(in_features=128, out_features=64, bias=True)
  (out_layer): Linear(in_features=64, out_features=10, bias=True)
)


In [20]:
### [테스트] 모델 사용 메모리 정보 확인
summary(model, input_size=(5000,784))

Layer (type:depth-idx)                   Output Shape              Param #
MnistMCFModel                            [5000, 10]                --
├─Linear: 1-1                            [5000, 64]                50,240
├─Linear: 1-2                            [5000, 128]               8,320
├─Linear: 1-3                            [5000, 256]               33,024
├─Linear: 1-4                            [5000, 512]               131,584
├─Linear: 1-5                            [5000, 128]               65,664
├─Linear: 1-6                            [5000, 64]                8,256
├─Linear: 1-7                            [5000, 10]                650
Total params: 297,738
Trainable params: 297,738
Non-trainable params: 0
Total mult-adds (G): 1.49
Input size (MB): 15.68
Forward/backward pass size (MB): 46.48
Params size (MB): 1.19
Estimated Total Size (MB): 63.35

In [21]:
class MnistDataset(Dataset):
    def __init__(self, featureDF, targetDF):
        self.featureDF=featureDF
        self.targetDF=targetDF
        self.n_rows=featureDF.shape[0]
        self.n_features=featureDF.shape[1]

    def __len__(self):
        return self.n_rows

    def __getitem__(self, index):
        # 텐서화
        featureTS=torch.FloatTensor(self.featureDF.iloc[index].values)
        targetTS=torch.FloatTensor(self.targetDF.iloc[index].values)

        # 피쳐와 타겟 반환
        return featureTS, targetTS

In [23]:
## [테스트] 데이터셋 인스턴스 생성

# - 커스텀데이터셋 인스턴스 생성
MnistDS=MnistDataset(X_train, y_train)

# - 데이터로더 인스턴스 생성
MinstDL = DataLoader(MnistDS)
for feature, label in MinstDL:
    print(feature.shape, label.shape, feature, label)
    break

torch.Size([1, 784]) torch.Size([1, 1]) tensor([[  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
           0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
           0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
           0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
           0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
           0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
           0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
           0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
           0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
           0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
           0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
           0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0., 

In [24]:
### 학습 진행 관련 설정
EPOCH = 1000
BATCH_SIZE = 10
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
LR = 0.001

In [28]:
# - 학습용, 검즈용, 테스트용 데이터셋
trainDS=MnistDataset(X_train, y_train)
valDS=MnistDataset(X_val, y_val)
testDS=MnistDataset(X_test, y_test)

# - 학습용 데이터로더 인스턴스
trainDL=DataLoader(MnistDS, batch_size=BATCH_SIZE)

In [29]:
# 최적화 인스턴스 => W,b 텐서 즉, model.parameters() 전달
optimizer=optim.Adam(model.parameters(), lr=LR)

# 손실함수 인스턴스 => 분류 => 다중분류 CrossEntropyLoss
#                            예측값은 선형식 결과 값 전달 ==> AF 처리 X
crossLoss=nn.CrossEntropyLoss()

In [33]:
## 학습의 효과 확인 손실값과 성능평가값 저장 필요
LOSS_HISTORY, SCORE_HISTORY=[[],[]], [[],[]]
CNT=len(trainDL)
print(f'BATCH_CNT => {CNT}')

for epoch in range(EPOCH):
    # 학습 모드로 모델 설정
    model.train()

    # 배치 크기 만큼 데이터 로딩해서 학습 진행
    loss_total, score_total=0,0
    for featureTS, targetTS in trainDL:
        # 학습 진행
        pre_y=model(featureTS)

        # 손실 계산 : nn.CrossEntropyLoss 요구사항 : 정답/타겟은 0D 또는 1D,  타입은 long
        loss=crossLoss(pre_y, targetTS.reshape(-1).long())
        loss_total += loss.item()

        # 성능평가 계산
        score=MulticlassF1Score(num_classes=10)(pre_y, targetTS.reshape(-1))
        score_total += score.item()

        # 최적화 진행
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # 에포크 당 검증기능
    # 모델 검증 모드 설정
    model.eval()
    with torch.no_grad():
        # 검증 데이터셋
        val_featrueTS=torch.FloatTensor(valDS.featureDF.values)
        val_targetTS=torch.FloatTensor(valDS.targetDF.values)

        # 추론/평가
        pre_val=model(val_featrueTS)
        print(pre_val.shape, val_targetTS.reshape(-1).shape)

        # 손실
        loss_val=crossLoss(pre_val, val_targetTS.reshape(-1).long())
        score_val=MulticlassF1Score(num_classes=10)(pre_val, val_targetTS.reshape(-1))

    # 에포크 당 손실값과 성능평가값 저장
    SCORE_HISTORY[0].append(score_total/CNT)
    LOSS_HISTORY[0].append(loss_total/CNT)

    LOSS_HISTORY[1].append(loss_val)
    SCORE_HISTORY[1].append(score_val)

    print(f'[{epoch}/{EPOCH}]\n- [TRAIN] LOSS : {LOSS_HISTORY[0][-1]} SCORE : {SCORE_HISTORY[0][-1]}')
    print(f'- [VALID] LOSS : {LOSS_HISTORY[1][-1]} SCORE : {SCORE_HISTORY[1][-1]}')

BATCH_CNT => 750
torch.Size([2500, 10]) torch.Size([2500])
[0/1000]
- [TRAIN] LOSS : 0.2703866273632739 SCORE : 0.8916125532388687
- [VALID] LOSS : 0.27713561058044434 SCORE : 0.9294871091842651
torch.Size([2500, 10]) torch.Size([2500])
[1/1000]
- [TRAIN] LOSS : 0.22699450400616236 SCORE : 0.9048979385693868
- [VALID] LOSS : 0.29668527841567993 SCORE : 0.9396421313285828
torch.Size([2500, 10]) torch.Size([2500])
[2/1000]
- [TRAIN] LOSS : 0.20082941282674924 SCORE : 0.917438595811526
- [VALID] LOSS : 0.35658347606658936 SCORE : 0.9281319975852966
torch.Size([2500, 10]) torch.Size([2500])
[3/1000]
- [TRAIN] LOSS : 0.20050404456942608 SCORE : 0.9200679570833842
- [VALID] LOSS : 0.25655391812324524 SCORE : 0.9520761966705322
torch.Size([2500, 10]) torch.Size([2500])
[4/1000]
- [TRAIN] LOSS : 0.16537155456283972 SCORE : 0.9363983329534531
- [VALID] LOSS : 0.270143985748291 SCORE : 0.954028844833374
torch.Size([2500, 10]) torch.Size([2500])
[5/1000]
- [TRAIN] LOSS : 0.11608733052146283 SCORE